In [ ]:
pip install tensorflow

In [ ]:

import numpy as np
import pandas as pd

# Sample user-item interaction data (replace with actual data)
# Columns: user_id, item_id, interaction (1 = interested, 0 = not interested)
data = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3],
    'item_id': [1, 2, 1, 3, 2, 3],
    'interaction': [1, 0, 1, 1, 0, 1]  # 1 = liked, 0 = not liked
})

# Create a user-item matrix
user_item_matrix = data.pivot(index='user_id', columns='item_id', values='interaction').fillna(0)

# Convert the user-item matrix to a NumPy array for input
user_item_matrix_np = user_item_matrix.values

# Number of users and items
num_users, num_items = user_item_matrix_np.shape

# Define the collaborative filtering model in TensorFlow (using matrix factorization)
class CollaborativeFilteringModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_size=10):
        super(CollaborativeFilteringModel, self).__init__()
        # Embedding layers for users and items
        self.user_embedding = tf.keras.layers.Embedding(num_users, embedding_size)
        self.item_embedding = tf.keras.layers.Embedding(num_items, embedding_size)

    def call(self, inputs):
        user, item = inputs
        user_embedded = self.user_embedding(user)
        item_embedded = self.item_embedding(item)
        return tf.reduce_sum(user_embedded * item_embedded, axis=1)  # Dot product

# Instantiate the model
model = CollaborativeFilteringModel(num_users, num_items)

# Compile the model with binary cross-entropy loss (since it's a binary classification problem: liked vs not liked)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Prepare the data
user_indices = data['user_id'].values - 1  # Subtract 1 to make 0-based index
item_indices = data['item_id'].values - 1  # Subtract 1 to make 0-based index
interactions = data['interaction'].values

# Train the model
model.fit([user_indices, item_indices], interactions, epochs=10, batch_size=2)

# Example: Get recommendations for user 1 (user_id = 1, which is index 0)
user_id = 1  # 1-based ID
user_items = np.arange(num_items)  # Get all item indices
predicted_scores = model.predict([np.full_like(user_items, user_id - 1), user_items])  # Subtract 1 for 0-based indexing

# Recommend top 3 items based on predicted score
recommended_items = user_items[np.argsort(predicted_scores.flatten())[-3:]]
print(f"Recommended items for User {user_id}: {recommended_items}")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.3677  
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.4668 
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.7952 
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.0144  
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.2221  
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.0189 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.8285  
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5.7845 
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.5254 
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.5887  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Recommended items for User 1: [0 2 1]


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Flatten, Input, Dot
from tensorflow.keras.optimizers import Adam

# Example Data: User-Item Interaction matrix
# This could represent whether a user liked a clothing item (1) or not (0).
# For a real application, you would load your data here.
data = {
    'user_id': [0, 0, 1, 1, 2, 2, 3, 3],
    'item_id': [0, 1, 1, 2, 0, 2, 0, 1],
    'rating': [1, 0, 1, 1, 0, 1, 1, 1]  # 1 = liked, 0 = not liked
}

df = pd.DataFrame(data)

# Number of unique users and items
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()

# Hyperparameters
embedding_size = 4  # You can change this based on the complexity of your data

# TensorFlow Model for Matrix Factorization
# Input layers
user_input = Input(shape=(1,), dtype=tf.int32, name='user')
item_input = Input(shape=(1,), dtype=tf.int32, name='item')

# Embedding layers for users and items
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size)(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size)(item_input)

# Flatten the embeddings
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

# Dot product of the user and item embeddings
dot_product = Dot(axes=1)([user_vec, item_vec])

# Create the model
model = Model(inputs=[user_input, item_input], outputs=dot_product)

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Prepare data for training (user, item pairs and ratings)
user_data = df['user_id'].values
item_data = df['item_id'].values
ratings = df['rating'].values

# Train the model
model.fit([user_data, item_data], ratings, epochs=50, batch_size=2)

# Now the model is trained and can make predictions
# For example, predict if user 0 likes item 2
user_id = np.array([0])
item_id = np.array([2])

predicted_rating = model.predict([user_id, item_id])
print(f"Predicted Rating (likelihood) for user 0 on item 2: {predicted_rating[0][0]}")



Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4333 - loss: 13.3079      
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3167 - loss: 9.6106  
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3667 - loss: 8.7667 
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2833 - loss: 10.0857
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2333 - loss: 11.4901     
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3167 - loss: 9.3129 
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5333 - loss: 10.5965     
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1333 - loss: 12.2279     
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3167 - loss: 7.9845 
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3667 - loss: 8.9248 
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4667 - loss: 6.9924     
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy